In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score

In [2]:
# train=pd.read_csv(r"C:\Users\User\Desktop\ml project\credit_defaulters_data.csv")
train=pd.read_csv("credit_defaulters_data.csv")
print(train.shape)

(211385, 8)


In [3]:
train.drop(['Unnamed: 0', 'Id'], axis=1, inplace=True)

In [4]:
train.head(5)

,age,profession,city,state,risk_flag,avg_income_per_annum
0,0.034483,0,1,1,0,0.040869
1,0.327586,1,1,0,0,0.040956
2,0.775862,1,1,1,0,0.032700
3,0.637931,0,0,0,0,0.010523
4,0.206897,0,0,0,0,0.055917


In [5]:
labels = train.pop('risk_flag')
labels

0         0
1         0
2         0
3         0
4         0
         ..
211380    1
211381    1
211382    1
211383    1
211384    1
Name: risk_flag, Length: 211385, dtype: int64

In [6]:
train.columns

Index(['age', 'profession', 'city', 'state', 'avg_income_per_annum'], dtype='object')

In [7]:
tr_cols=list(train.columns)
tr_cols.append('risk_flag')
tr_cols=np.array(tr_cols)
tr_cols

array(['age', 'profession', 'city', 'state', 'avg_income_per_annum',
       'risk_flag'], dtype='<U20')

In [8]:
def data_splicing(train, labels, tr_cols, target):
    # Firstly, make d0 and d1 as risk_flag specific datasets
    d = pd.DataFrame(np.hstack((train.values, labels.values.reshape(-1, 1))), columns=tr_cols)
    d0 = d[d[target]==0]
    d1 = d[d[target]==1]

    # Randomized Data Splicing
    tr_0, ts_0 = train_test_split(d0.values, test_size=3000, random_state=None)
    tr_1, ts_1 = train_test_split(d1.values, test_size=3000, random_state=None)
    # # Validation Data
    # tr_0, trv_0 = train_test_split(tr_0, test_size=1000, random_state=None)
    # tr_1, trv_1 = train_test_split(tr_1, test_size=1000, random_state=None)

    print("Train 0, 1:", tr_0.shape, tr_1.shape)

    X_tr = np.concatenate((tr_0[:, :tr_0.shape[1]-1], tr_1[:, :tr_1.shape[1]-1]), axis=0)
    print(X_tr)
    y_tr = np.concatenate((tr_0[:, tr_0.shape[1]-1:], tr_1[:, tr_1.shape[1]-1:]), axis=0)
    y_tr = y_tr.ravel()
    print(y_tr.shape)

    X_ts = np.concatenate((ts_0[:, :ts_0.shape[1]-1], ts_1[:, :ts_1.shape[1]-1]), axis=0)
    print(X_ts)
    y_ts = np.concatenate((ts_0[:, ts_0.shape[1]-1:], ts_1[:, ts_1.shape[1]-1:]), axis=0)
    y_ts = y_ts.ravel()
    print(y_ts.shape)
    
    return tr_0, tr_1, X_ts, y_ts
    # X_trv = np.concatenate((trv_0[:, :trv_0.shape[1]-1], trv_1[:, :trv_1.shape[1]-1]), axis=0)
    # print(X_trv)
    # y_trv = np.concatenate((trv_0[:, trv_0.shape[1]-1:], trv_1[:, trv_1.shape[1]-1:]), axis=0)
    # y_trv = y_trv.ravel()
    # print(y_trv)

In [9]:
def rf_model_team_trainer(tr_0, tr_1):
    
    rf_models = []

    for i in range(5):

        tr_0n, trv_0n = train_test_split(tr_0, test_size=2000, random_state=None)
        tr_1n, trv_1n = train_test_split(tr_1, test_size=2000, random_state=None)

        X_tr = np.concatenate((tr_0n[:, :tr_0n.shape[1]-1], tr_1n[:, :tr_1n.shape[1]-1]), axis=0)
        y_tr = np.concatenate((tr_0n[:, tr_0n.shape[1]-1:], tr_1n[:, tr_1n.shape[1]-1:]), axis=0)
        y_tr = y_tr.ravel()

        X_trv = np.concatenate((trv_0n[:, :trv_0n.shape[1]-1], trv_1n[:, :trv_1n.shape[1]-1]), axis=0)
        y_trv = np.concatenate((trv_0n[:, trv_0n.shape[1]-1:], trv_1n[:, trv_1n.shape[1]-1:]), axis=0)
        y_trv = y_trv.ravel()

        rf = RandomForestRegressor(n_jobs=-1, random_state=0)
        rf.fit(X_tr, y_tr)
        
        def tes(a):
            if a>0.05:
                return 1
            return 0
        def rfpre(a):
            p = rf.predict(a)
            p=list(map(tes,p))
            return p

        ac = accuracy_score(y_trv, rfpre(X_trv))
        print(f"Accuracy: {ac}")
        if(ac>=0.88):
            rf_models.append(rf)
            print("Model", i+1, "--> Done")
        else:
            print("Less Accurate Model")
            
    return rf_models
            

In [10]:
def rf_team_predictions(rf_models, test_x):
    
    predicted_flags=[]
    
    def tes(a):
        if a>0.05:
            return 1
        return 0
        
    def rfpre(a, rf):
        p = rf.predict(a)
        p=list(map(tes,p))
        return p
    
    for model in rf_models:
        predicted_flags.append(np.array(rfpre(test_x, model)).astype('int64'))

#     predicted_flags/=len(rf_models)
    predicted_flags = np.array(predicted_flags)
    final_prediction = np.zeros(predicted_flags.shape[1])
    for i in range(predicted_flags.shape[1]):
        final_prediction[i] = np.bincount(predicted_flags[:, i].ravel()).argmax()
    
    return final_prediction.astype('int64')

In [11]:
tr_0, tr_1, X_ts, y_ts = data_splicing(train, labels, tr_cols, 'risk_flag')

Train 0, 1: (177389, 6) (27996, 6)
[[3.96551724e-01 1.00000000e+00 0.00000000e+00 0.00000000e+00
  3.99249916e-03]
 [7.41379310e-01 0.00000000e+00 1.00000000e+00 0.00000000e+00
  8.58953852e-04]
 [1.55172414e-01 0.00000000e+00 0.00000000e+00 1.00000000e+00
  8.57298550e-03]
 ...
 [3.44827586e-02 0.00000000e+00 1.00000000e+00 1.00000000e+00
  3.35979412e-02]
 [7.06896552e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
  9.16291118e-03]
 [9.13793103e-01 0.00000000e+00 1.00000000e+00 0.00000000e+00
  1.90260284e-02]]
(205385,)
[[0.0862069  1.         0.         0.         0.03295396]
 [0.10344828 1.         1.         0.         0.03101355]
 [0.46551724 0.         0.         0.         0.00664868]
 ...
 [0.10344828 0.         1.         1.         0.0129371 ]
 [0.29310345 1.         0.         0.         0.04077592]
 [0.46551724 1.         0.         0.         0.00362929]]
(6000,)


In [12]:
rf_models = rf_model_team_trainer(tr_0, tr_1)

Accuracy: 0.88225
Model 1 --> Done
Accuracy: 0.893
Model 2 --> Done
Accuracy: 0.8755
Less Accurate Model
Accuracy: 0.8825
Model 4 --> Done
Accuracy: 0.88675
Model 5 --> Done


In [13]:
predicted_flags = rf_team_predictions(rf_models, X_ts)
print(classification_report(y_ts, predicted_flags))
auc = roc_auc_score(y_ts, predicted_flags)
print('ROC AUC: %f' % auc)
print(np.unique(predicted_flags))

print(confusion_matrix(y_ts, predicted_flags))

              precision    recall  f1-score   support

         0.0       0.93      0.82      0.87      3000
         1.0       0.84      0.94      0.88      3000

    accuracy                           0.88      6000
   macro avg       0.88      0.88      0.88      6000
weighted avg       0.88      0.88      0.88      6000

ROC AUC: 0.877167
[0 1]
[[2450  550]
 [ 187 2813]]


In [14]:
tr_cols

array(['age', 'profession', 'city', 'state', 'avg_income_per_annum',
       'risk_flag'], dtype='<U20')

In [15]:
predicted_flags = rf_team_predictions(rf_models, train.values)
print(classification_report(labels.values, predicted_flags))
auc = roc_auc_score(labels.values, predicted_flags)
print('ROC AUC: %f' % auc)

              precision    recall  f1-score   support

           0       1.00      0.86      0.92    180389
           1       0.55      0.99      0.71     30996

    accuracy                           0.88    211385
   macro avg       0.77      0.93      0.82    211385
weighted avg       0.93      0.88      0.89    211385

ROC AUC: 0.926421


In [16]:
print(confusion_matrix(labels.values, predicted_flags))

[[155176  25213]
 [   229  30767]]


In [17]:
train_new = train.copy()

In [18]:
train_new['predicted_values'] = predicted_flags
train_new['risk_flag'] = labels
train_new

,age,profession,city,state,avg_income_per_annum,predicted_values,risk_flag
0,0.034483,0,1,1,0.040869,0,0
1,0.327586,1,1,0,0.040956,0,0
2,0.775862,1,1,1,0.032700,0,0
3,0.637931,0,0,0,0.010523,0,0
4,0.206897,0,0,0,0.055917,0,0
...,...,...,...,...,...,...,...
211380,0.241379,0,1,1,0.005113,1,1
211381,0.724138,0,1,1,0.002389,1,1
211382,0.448276,1,0,0,0.041340,1,1
211383,0.534483,0,0,1,0.066190,1,1


In [19]:
train_new = train_new.assign(incorrect_ones=0)
train_new

,age,profession,city,state,avg_income_per_annum,predicted_values,risk_flag,incorrect_ones
0,0.034483,0,1,1,0.040869,0,0,0
1,0.327586,1,1,0,0.040956,0,0,0
2,0.775862,1,1,1,0.032700,0,0,0
3,0.637931,0,0,0,0.010523,0,0,0
4,0.206897,0,0,0,0.055917,0,0,0
...,...,...,...,...,...,...,...,...
211380,0.241379,0,1,1,0.005113,1,1,0
211381,0.724138,0,1,1,0.002389,1,1,0
211382,0.448276,1,0,0,0.041340,1,1,0
211383,0.534483,0,0,1,0.066190,1,1,0


In [20]:
incorrect_pred_indices = train_new[train_new.predicted_values != train_new.risk_flag].index

In [21]:
train_new.loc[incorrect_pred_indices, 'incorrect_ones'] = 1
train_new

,age,profession,city,state,avg_income_per_annum,predicted_values,risk_flag,incorrect_ones
0,0.034483,0,1,1,0.040869,0,0,0
1,0.327586,1,1,0,0.040956,0,0,0
2,0.775862,1,1,1,0.032700,0,0,0
3,0.637931,0,0,0,0.010523,0,0,0
4,0.206897,0,0,0,0.055917,0,0,0
...,...,...,...,...,...,...,...,...
211380,0.241379,0,1,1,0.005113,1,1,0
211381,0.724138,0,1,1,0.002389,1,1,0
211382,0.448276,1,0,0,0.041340,1,1,0
211383,0.534483,0,0,1,0.066190,1,1,0


In [22]:
train_new.describe()

,age,profession,city,state,avg_income_per_annum,predicted_values,risk_flag,incorrect_ones
count,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000,211385.000000
mean,0.520233,0.488545,0.499208,0.468311,0.027180,0.264825,0.146633,0.120359
std,0.290568,0.499870,0.500001,0.498996,0.045728,0.441241,0.353740,0.325381
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.275862,0.000000,0.000000,0.000000,0.009094,0.000000,0.000000,0.000000
50%,0.534483,0.000000,0.000000,0.000000,0.018391,0.000000,0.000000,0.000000
75%,0.775862,1.000000,1.000000,1.000000,0.032165,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
train_new.drop(['predicted_values'], axis=1, inplace=True)

In [24]:
train_new.to_csv('credit_defaulters_data_with_fail_cases.csv', index_label=False)
print('Complete')

Complete
